In [1]:
%matplotlib inline
import torch
import torch.nn as nn
# import pandas as pd
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import copy

In [3]:
class Model1(torch.nn.Module):
    def forward(self, x):
        return add_one(x)

In [4]:
torch.onnx.export(Model1(), torch.zeros(1), 'example.onnx', input_names=['input_data'])

In [5]:
import caffe2.python.onnx.backend as backend
import numpy as np
import onnx

In [6]:
model = onnx.load('example.onnx')
onnx.checker.check_model(model)
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %input_data[FLOAT, 1]\n) {\n  %1 = Constant[value = <Scalar Tensor []>]()\n  %2 = Add(%input_data, %1)\n  return %2\n}'

In [7]:
rep = backend.prepare(model, device='CUDA:0') # Looks like the model has to be saved correctly
rep.run(torch.FloatTensor([1]).numpy())

Outputs(_0=array([2.], dtype=float32))

# More Complicated DNN

In [88]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(1, 2),
            nn.Linear(2, 1)
        )
    def forward(self, x):
        return self.model(x)

In [89]:
loss = torch.nn.MSELoss()
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.2, betas=(0.9, 0.999))

In [90]:
for i in range(100):
    optimizer.zero_grad()
    y = torch.FloatTensor([10])
    y_h = model(torch.FloatTensor([23]))
    loss(y, y_h).backward()
    optimizer.step()

In [91]:
model(torch.FloatTensor([23]))

tensor([10.0210], grad_fn=<AddBackward0>)

In [92]:
x = torch.FloatTensor([0])
torch.onnx.export(model, (x), 'example.onnx', input_names=['input_data'])

In [93]:
import caffe2.python.onnx.backend as backend
import numpy as np
import onnx

In [94]:
model = onnx.load('example.onnx')
onnx.checker.check_model(model)
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %input_data[FLOAT, 1]\n) initializers (\n  %11[FLOAT, 1x2]\n  %12[FLOAT, 2x1]\n  %model.0.bias[FLOAT, 2]\n  %model.1.bias[FLOAT, 1]\n) {\n  %6 = MatMul(%input_data, %11)\n  %7 = Add(%6, %model.0.bias)\n  %9 = MatMul(%7, %12)\n  %10 = Add(%9, %model.1.bias)\n  return %10\n}'

In [95]:
import onnxruntime as ort
ort_sess = ort.InferenceSession('example.onnx')
print(ort_sess.run(None, {'input_data': torch.FloatTensor([23]).numpy()}))

# Don't Try to Run with the Following
# rep = backend.prepare(model, device='CUDA:0') # Looks like the model has to be saved correctly
# rep.run(torch.FloatTensor([1]).numpy())

[array([10.020969], dtype=float32)]
